
# Shared rides with pricing - Delft 

 ### Choice Function (Deterministic):
 `pool_price.py`
 * Pickup Distance: distance from driver initial position to the first pickup point
 * Travel Distance: distance from driver's initial position to the drop off point of the last passenger
 * Operating Cost: This include all the expenses
 * Profit: Driver revenue to serve the request
            
 
  ### KPI:
   
   * Profit of Individual driver
   * Profit of all the drivers
   * No.of rejected rides
   * U - PAX (Utility) 
  
   ### TBD- Choice Function (Probablistic):
  
  * choice logic to be applied inside `pool_price.py` 
  * P(R)= exp(beta * Profit_R)/ sum_all the rides( exp(beta * Profit_R)
 



-------------------------------------------------------------------------------------------------------

# Pricing and Driver Earnings for a Two-Sided Mobility Platform: A Case of Amsterdam, the Netherlands

or 

# The Effects of Profit-Based Pricing on Driver Earnings and Performance of Two-Sided Mobility Platforms

# Abstract  

In this paper, we investigate how the  pricing of ride-pooling affects driver earnings. We also examine how profit-based setting affects these performance indicators. To this end, we applied a matching algorithm  to the case of ride-pooling and give a choice set to the driver for the case of Amsterdam, the Netherlands. For our simulation, we utilize an agent-based simulator reproducing the transport systems for two-sided mobility platforms (like Uber and Lyft) and applied three state-of-the-art pricing strategies such as <strong>profit maximization</strong>,  <strong>solo ride-hailing</strong>, and <strong>nearest pickup ride-pooling</strong>. We find that the profit maximization pricing strategy outperforms the other and traveler utility can be further improved by $\%X$ while reducing the total cost to serve the pooled rides. While offering a discount for profit maximization travel time is significantly higher $\%X$  than for private rides. 

-------------------------------------------------------------------------------------------------------

## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo ride-hailing</strong>

<strong>3. Nearest pickup ride-pooling</strong>

## Load ExMAS and MaaSSim


In [1]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging
import matplotlib.pyplot as plt

import pandas as pd
import ExMAS

## Delft, Netherlands

In [2]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 1 # 6 minutes hour of simulation
params.nP = 100 # reuqests (and passengers)
params.nV = 5 # vehicles



## Parameters for ExMAS

In [3]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.5
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



In [4]:
inData = ExMAS.main(inData, params.shareability, plot=False) # create shareability graph (ExMAS) 

09-03-23 11:40:18-INFO-Initializing pairwise trip shareability between 100 and 100 trips.
09-03-23 11:40:18-INFO-creating combinations
09-03-23 11:40:18-INFO-9900	 nR*(nR-1)
09-03-23 11:40:18-INFO-Reduction of feasible pairs by 99.37%
09-03-23 11:40:18-INFO-Degree 2 	Completed
09-03-23 11:40:18-INFO-trips to extend at degree 2 : 196
09-03-23 11:40:18-INFO-At degree 2 feasible extensions found out of 27 searched
09-03-23 11:40:18-INFO-Degree 3 	Completed
09-03-23 11:40:18-INFO-trips to extend at degree 3 : 27
09-03-23 11:40:18-INFO-At degree 3 feasible extensions found out of 0 searched
09-03-23 11:40:18-INFO-Degree 4 	Completed
09-03-23 11:40:18-INFO-Max degree reached 4
09-03-23 11:40:18-INFO-Trips still possible to extend at degree 4 : 0


In [5]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

09-03-23 11:40:18-INFO-Matching 100 trips to 323 rides in order to min u_veh
09-03-23 11:40:18-INFO-Problem solution: Optimal. 
Total costs for single trips:         26,470 
reduced by matching to:               22,137


In [6]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],4.791500,259,1,[4.7915],"[0, 259]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 5400602513, 1569646491]","[None, 0, 0]",node time req_id od 0 ...,259,2.59,3.885000,0.777000,3.1080
1,[1],0.567000,30,1,[0.5670000000000001],"[10, 30]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44833176, 44831868]","[None, 1, 1]",node time req_id od 0 NaN...,30,0.30,0.462000,0.092400,0.3696
2,[2],3.505500,189,1,[3.5054999999999996],"[139, 189]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 643586473, 4244384221]","[None, 2, 2]",node time req_id od 0 ...,189,1.89,2.844000,0.568800,2.2752
3,[3],0.900000,48,1,[0.9],"[156, 48]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44848435, 44849727]","[None, 3, 3]",node time req_id od 0 NaN...,48,0.48,0.732000,0.146400,0.5856
4,[4],6.902000,373,1,[6.902],"[295, 373]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1620839921, 44845277]","[None, 4, 4]",node time req_id od 0 ...,373,3.73,5.596500,1.119300,4.4772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,"[87, 85, 93]",16.436550,687,30,"[5.065475, 4.65555, 6.7155249999999995]","[2976.0, 58, 202, 79, 17, 331]","[87, 85, 93]","[85, 87, 93]",3,318,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44882305, 3669218076, 484705811, 448220...","[None, 87, 85, 93, 85, 87, 93]",node time req_id od 0 ...,687,6.87,10.676250,2.135250,8.5410
319,"[39, 37, 49]",16.286800,785,30,"[6.0733, 4.3328500000000005, 5.88065]","[1714.5, 103, 277, 48, 33, 324]","[39, 37, 49]","[37, 39, 49]",3,319,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44715431, 44731573, 3581094285, 4300758...","[None, 39, 37, 49, 37, 39, 49]",node time req_id od 0 ...,785,7.85,10.333125,2.066625,8.2665
320,"[39, 37, 44]",13.629750,491,30,"[5.85945, 3.98705, 3.78325]","[1714.5, 103, 86, 200, 33, 69]","[39, 37, 44]","[37, 39, 44]",3,320,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44715431, 44731573, 1402734813, 4300758...","[None, 39, 37, 44, 37, 39, 44]",node time req_id od 0 ...,491,4.91,8.886375,1.777275,7.1091
321,"[39, 37, 46]",16.870725,677,30,"[5.955, 4.0826, 6.833125]","[1714.5, 103, 134, 173, 33, 234]","[39, 37, 46]","[37, 39, 46]",3,321,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44715431, 44731573, 1571087635, 4300758...","[None, 39, 37, 46, 37, 39, 46]",node time req_id od 0 ...,677,6.77,11.322000,2.264400,9.0576


In [7]:
params.kpi = 1

In [8]:
sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate

09-03-23 11:40:22-INFO-Matching 100 trips to 323 rides in order to min u_veh
09-03-23 11:40:22-INFO-Problem solution: Optimal. 
Total costs for single trips:         26,470 
reduced by matching to:               22,137
0.5
09-03-23 11:40:24-CRITICAL-this is request 0 with [0, 191] available rides.
09-03-23 11:40:24-CRITICAL-ride 0 available [1]
09-03-23 11:40:24-CRITICAL-ride 191 available [1, 0]
09-03-23 11:40:24-CRITICAL-this is reuqest 0 with [0, 191] still available rides.
Profit Maximization
09-03-23 11:40:24-CRITICAL-vehicle 1 has 2 choices
0.5
0.5
09-03-23 11:40:24-CRITICAL-this is request 2 with [2, 191] available rides.
09-03-23 11:40:24-CRITICAL-ride 2 available [1]
09-03-23 11:40:24-CRITICAL-ride 191 not available [5, 1]
09-03-23 11:40:24-CRITICAL-this is reuqest 2 with [2] still available rides.
0.5
0.5
09-03-23 11:40:24-CRITICAL-this is request 4 with [4, 148, 149, 150, 151] available rides.
09-03-23 11:40:24-CRITICAL-ride 4 available [1]
09-03-23 11:40:24-CRITICAL-ride 14

In [9]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,REPOSITIONED,DECIDES_NOT_TO_DRIVE,ENDS_SHIFT,TRAVEL,WAIT,CRUISE,OPERATIONS,IDLE,PAX_KM,REVENUE
veh,,,,,,,,,,,,,,,,,,,,,
1,14,0.0,False,0,0,70.0,15.0,0,279.0,0,...,0,0,6209.0,2041.0,0,1765.0,714.0,1689.0,33.592,3.4128
2,10,0.0,False,0,0,236.0,25.0,0,160.0,0,...,0,0,7642.0,1502.0,0,934.0,485.0,4721.0,20.273,6.9840
3,6,0.0,False,0,0,405.0,25.0,0,237.0,0,...,0,0,8032.0,1126.0,0,794.0,442.0,5670.0,18.259,7.1010
4,5,0.0,False,0,0,325.0,15.0,0,257.0,0,...,0,0,8649.0,1016.0,0,387.0,422.0,6824.0,16.343,0.4896
5,11,0.0,False,0,0,205.0,30.0,0,160.0,0,...,0,0,6828.0,1569.0,0,1677.0,520.0,3062.0,27.313,4.7220
